In [1]:
import yt
import firefly.dataParser
import firefly.snapshot_utils

In [2]:
import os
def load_AHF(directory,snapnum,current_redshift,hubble = 0.702):
        path = os.path.join(directory,'../halo/ahf/halo_00000_smooth.dat')
        
        ## find column numbers without having to count
        names_to_read = ['snum','Xc','Yc','Zc','Rvir','v_esc','Rstar0.5']
        
        ## load the first line of the datafile
        names = list(np.genfromtxt(path,skip_header=0,max_rows = 1,dtype=str))
        cols = []

        ## find the column each name appears in
        for name in names_to_read:
            cols+=[names.index(name)]

        ## load the rest of the file
        sns,xs,ys,zs, rvirs, vescs, rstar_halfs = np.genfromtxt(
            path,delimiter='\t',usecols=cols,unpack=1,skip_header=1)

        ## which row do I care about? make an index array
        index = sns==snapnum
        if np.sum(index)==0:
            ## snapnum is not in this halo file
            raise IOError
            
        ## presumably in comoving kpc/h 
        halo_center = np.array([xs[index],ys[index],zs[index]])#/hubble*(1/(1+current_redshift))
        halo_center = halo_center.reshape(3,)

        ## convert other quantities one might care about from comoving kpc to pkpc
        rvir = rvirs[index][0]/hubble/(1+current_redshift)
        vesc = vescs[index][0]
        rstar_half = rstar_halfs[index][0]/hubble/(1+current_redshift)
        return halo_center, rvir, vesc, rstar_half

In [191]:
halo_center,rvir,vesc,rstar_half = load_AHF(
    "/Users/agurvich/research/snaps/m12i_res7100/output/",
    600,
    0)

In [192]:
ds = yt.load("/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5")

yt : [INFO     ] 2019-03-04 16:34:46,337 Calculating time from 1.000e+00 to be 4.355e+17 seconds
yt : [INFO     ] 2019-03-04 16:34:46,339 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2019-03-04 16:34:46,402 Parameters: current_time              = 4.3545571088051386e+17 s
yt : [INFO     ] 2019-03-04 16:34:46,403 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2019-03-04 16:34:46,404 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-03-04 16:34:46,406 Parameters: domain_right_edge         = [60000. 60000. 60000.]
yt : [INFO     ] 2019-03-04 16:34:46,407 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2019-03-04 16:34:46,408 Parameters: current_redshift          = 0.0
yt : [INFO     ] 2019-03-04 16:34:46,409 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2019-03-04 16:34:46,410 Parameters: omega_matter              = 0.272
yt : [INFO     ] 2019-03-04 16:34:46,411 Parameters: omega_radiation           = 0.

In [204]:
var = ds.particle_unions['all']

In [207]:
ds.field_list

[('PartType0', 'Coordinates'),
 ('PartType0', 'Density'),
 ('PartType0', 'ElectronAbundance'),
 ('PartType0', 'InternalEnergy'),
 ('PartType0', 'Masses'),
 ('PartType0', 'Metallicity_00'),
 ('PartType0', 'Metallicity_01'),
 ('PartType0', 'Metallicity_02'),
 ('PartType0', 'Metallicity_03'),
 ('PartType0', 'Metallicity_04'),
 ('PartType0', 'Metallicity_05'),
 ('PartType0', 'Metallicity_06'),
 ('PartType0', 'Metallicity_07'),
 ('PartType0', 'Metallicity_08'),
 ('PartType0', 'Metallicity_09'),
 ('PartType0', 'Metallicity_10'),
 ('PartType0', 'Metallicity_11'),
 ('PartType0', 'Metallicity_12'),
 ('PartType0', 'Metallicity_13'),
 ('PartType0', 'Metallicity_14'),
 ('PartType0', 'Metallicity_15'),
 ('PartType0', 'Metallicity_16'),
 ('PartType0', 'NeutralHydrogenAbundance'),
 ('PartType0', 'ParticleIDs'),
 ('PartType0', 'Potential'),
 ('PartType0', 'SmoothingLength'),
 ('PartType0', 'StarFormationRate'),
 ('PartType0', 'Velocities'),
 ('PartType1', 'Coordinates'),
 ('PartType1', 'Masses'),
 ('P

In [5]:
sphere = ds.sphere(halo_center,(15,'kpc'))

yt : [INFO     ] 2019-03-04 14:40:14,346 Allocating for 1.467e+08 particles (index particle type 'all')
yt : [INFO     ] 2019-03-04 14:40:42,560 Identified 8.952e+06 octs


In [149]:
output_region_to_firefly(sphere,write_files=1)

Coordinates detected
Density detected
ElectronAbundance detected
InternalEnergy detected
Masses detected
Metallicity_00 detected
Metallicity_01 detected
Metallicity_02 detected
Metallicity_03 detected
Metallicity_04 detected
Metallicity_05 detected
Metallicity_06 detected
Metallicity_07 detected
Metallicity_08 detected
Metallicity_09 detected
Metallicity_10 detected
Metallicity_11 detected
Metallicity_12 detected
Metallicity_13 detected
Metallicity_14 detected
Metallicity_15 detected
Metallicity_16 detected
NeutralHydrogenAbundance detected
ParticleIDs detected
Potential detected
SmoothingLength detected
StarFormationRate detected
Velocities detected
Coordinates detected
Masses detected
ParticleIDs detected
Potential detected
Velocities detected
Coordinates detected
Masses detected
Metallicity_00 detected
Metallicity_01 detected
Metallicity_02 detected
Metallicity_03 detected
Metallicity_04 detected
Metallicity_05 detected
Metallicity_06 detected
Metallicity_07 detected
Metallicity_08 

In [190]:
ramses_ds.all_data().ds.particle_fields_by_type

defaultdict(list,
            {'io': ['particle_extra_field_1',
              'particle_extra_field_2',
              'particle_identity',
              'particle_mass',
              'particle_position_x',
              'particle_position_y',
              'particle_position_z',
              'particle_refinement_level',
              'particle_velocity_x',
              'particle_velocity_y',
              'particle_velocity_z']})

In [133]:
ramses_ds = yt.load("/Users/agurvich/Desktop/yt_workshop/DICEGalaxyDisk_nonCosmological/output_00002/info_00002.txt")

yt : [INFO     ] 2019-03-04 15:59:07,883 Parameters: current_time              = 0.0299468077820411
yt : [INFO     ] 2019-03-04 15:59:07,884 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2019-03-04 15:59:07,884 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-03-04 15:59:07,886 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2019-03-04 15:59:07,887 Parameters: cosmological_simulation   = 0


In [172]:
ramses_ds = yt.load("/Users/agurvich/Desktop/yt_workshop/output_00080/info_00080.txt")

yt : [INFO     ] 2019-03-04 16:22:19,722 Parameters: current_time              = 1.119216564055017
yt : [INFO     ] 2019-03-04 16:22:19,723 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2019-03-04 16:22:19,724 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-03-04 16:22:19,725 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2019-03-04 16:22:19,726 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2019-03-04 16:22:19,727 Parameters: current_redshift          = 0.14255728632206321
yt : [INFO     ] 2019-03-04 16:22:19,728 Parameters: omega_lambda              = 0.723999977111816
yt : [INFO     ] 2019-03-04 16:22:19,729 Parameters: omega_matter              = 0.276000022888184
yt : [INFO     ] 2019-03-04 16:22:19,730 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2019-03-04 16:22:19,730 Parameters: hubble_constant           = 0.703000030517578


In [178]:
region = ramses_ds.sphere(ramses_ds.domain_center,(1000,'kpc'))`

In [223]:
output_region_to_firefly(ramses_ds.all_data(),fields_to_include=['particle_extra_field_1','particle_extra_field_2'],write_files=1)

detected (but did not request) io particle_identity
detected (but did not request) io particle_mass
detected (but did not request) io particle_refinement_level
outputting Particle Group of io
Contains 1090895 (1090895 after dec) particles and 3 arrays

['Velocities' 'particle_extra_field_1' 'particle_extra_field_2'] filter: [1. 1. 1.] colormap: [1. 1. 1.]


dict_keys([])

In [19]:
ds = yt.load("ahf_halos/snap_N64L16_062.parameter")

yt : [INFO     ] 2019-03-05 09:18:30,748 Parameters: current_time              = 3.3488996201228216e+16 s
yt : [INFO     ] 2019-03-05 09:18:30,749 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2019-03-05 09:18:30,750 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-03-05 09:18:30,751 Parameters: domain_right_edge         = [16000. 16000. 16000.]
yt : [INFO     ] 2019-03-05 09:18:30,752 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2019-03-05 09:18:30,752 Parameters: current_redshift          = 4.0
yt : [INFO     ] 2019-03-05 09:18:30,753 Parameters: omega_lambda              = 0.7
yt : [INFO     ] 2019-03-05 09:18:30,754 Parameters: omega_matter              = 0.3
yt : [INFO     ] 2019-03-05 09:18:30,755 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2019-03-05 09:18:30,755 Parameters: hubble_constant           = 1.0


In [33]:
ds.field_list

[('all', 'Eax'),
 ('all', 'Eay'),
 ('all', 'Eaz'),
 ('all', 'Ebx'),
 ('all', 'Eby'),
 ('all', 'Ebz'),
 ('all', 'Ecx'),
 ('all', 'Ecy'),
 ('all', 'Ecz'),
 ('all', 'Ekin'),
 ('all', 'Epot'),
 ('all', 'ID'),
 ('all', 'Lx'),
 ('all', 'Ly'),
 ('all', 'Lz'),
 ('all', 'Mvir'),
 ('all', 'Phi0'),
 ('all', 'Rmax'),
 ('all', 'Rvir'),
 ('all', 'SurfP'),
 ('all', 'VXc'),
 ('all', 'VYc'),
 ('all', 'VZc'),
 ('all', 'Vmax'),
 ('all', 'Xc'),
 ('all', 'Yc'),
 ('all', 'Zc'),
 ('all', 'b'),
 ('all', 'c'),
 ('all', 'cNFW'),
 ('all', 'com_offset'),
 ('all', 'fMhires'),
 ('all', 'hostHalo'),
 ('all', 'lambda'),
 ('all', 'lambdaE'),
 ('all', 'mbp_offset'),
 ('all', 'nbins'),
 ('all', 'npart'),
 ('all', 'numSubStruct'),
 ('all', 'ovdens'),
 ('all', 'r2'),
 ('all', 'sigV'),
 ('all', 'v_esc'),
 ('halos', 'Eax'),
 ('halos', 'Eay'),
 ('halos', 'Eaz'),
 ('halos', 'Ebx'),
 ('halos', 'Eby'),
 ('halos', 'Ebz'),
 ('halos', 'Ecx'),
 ('halos', 'Ecy'),
 ('halos', 'Ecz'),
 ('halos', 'Ekin'),
 ('halos', 'Epot'),
 ('halos', 

In [20]:
all_halos = ds.all_data()

In [18]:
all_halos.field_parameters

{'center': YTArray([8000., 8000., 8000.]) code_length,
 'bulk_velocity': YTArray([0., 0., 0.]) cm/s,
 'bulk_magnetic_field': YTArray([0., 0., 0.]) G,
 'normal': YTArray([0., 0., 1.]) (dimensionless)}

In [14]:
all_halos.get_data()

In [59]:
output_halo_catalog_to_firefly(all_halos,fields_to_include=['Mvir'],write_files=1)

detected (but did not request) halos Eax
detected (but did not request) halos Eay
detected (but did not request) halos Eaz
detected (but did not request) halos Ebx
detected (but did not request) halos Eby
detected (but did not request) halos Ebz
detected (but did not request) halos Ecx
detected (but did not request) halos Ecy
detected (but did not request) halos Ecz
detected (but did not request) halos Ekin
detected (but did not request) halos Epot
detected (but did not request) halos ID
detected (but did not request) halos Lx
detected (but did not request) halos Ly
detected (but did not request) halos Lz
detected (but did not request) halos Phi0
detected (but did not request) halos Rmax
detected (but did not request) halos Rvir
detected (but did not request) halos SurfP
detected (but did not request) halos Vmax
detected (but did not request) halos b
detected (but did not request) halos c
detected (but did not request) halos cNFW
detected (but did not request) halos com_offset
detected

In [84]:
def quick_output_region_to_firefly(
    region,
    fields_to_include= [],
    fields_units = [],
    write_files=0):
    
    assert len(fields_units) == len(fields_to_include)
    
    ## initialize a firefly reader instance
    reader = firefly.dataParser.Reader(
    JSONdir="/Users/agurvich/research/repos/Firefly/data/yt",
    write_startup=1,
    prefix='ytData',
    clean_JSONdir=True
    )

    ## create a ParticleGroup object that contains *every* field
    for ptype in sorted(region.ds.particle_types_raw):
        ## skip this particle type if it has no particles in this region!
        if region[ptype,'relative_particle_position'].shape[0]==0:
            continue
        
        ## read the available extra fields from yt
        this_ptype_fields = region.ds.particle_fields_by_type[ptype]
        
        ## define which fields we want to save
        tracked_arrays = [region[ptype,'relative_particle_velocity'].convert_to_units('km/s')]
        tracked_names = ['Velocities']
        
        ## load the extra fields
        for field in this_ptype_fields:
            if 'position' in field or 'velocity' in field:
                continue
            if field not in fields_to_include:
                print('detected (but did not request)',ptype,field)
                continue

        ## explicitly go after the fields we want
        for field,units in zip(fields_to_include,fields_units):
            log_flag = 'log(' in units
            
            if field not in tracked_names:
                try:
                    this_field_array = region[ptype,field]
                    if log_flag:
                        units = units[len('log('):-1]
                        field = 'log{}'.format(field)
                        
                    this_field_array.convert_to_units(units)
                    
                    if log_flag:
                        this_field_array = np.log10(this_field_array)
                    tracked_arrays +=[this_field_array]
                    tracked_names = np.append(tracked_names,[field],axis=0)
                except:
                    raise
        
        ## flag whether we want to filter and/or color by these fields
        tracked_filter_flags = np.ones(len(tracked_names))
        tracked_colormap_flags = np.ones(len(tracked_names))
                           

        ## create a firefly ParticleGroup for this particle type
        pg = firefly.dataParser.ParticleGroup(
            UIname =  ptype,
            coordinates=region[ptype,'relative_particle_position'].convert_to_units('kpc'),
            tracked_arrays=tracked_arrays,
            tracked_names=tracked_names,
            tracked_filter_flags=tracked_filter_flags,
            tracked_colormap_flags=tracked_colormap_flags,
            decimation_factor=1
            )
        
        ## bind this particle group to the firefly reader object
        reader.addParticleGroup(pg)
    if write_files:
        reader.dumpToJSON(loud=0)

In [85]:
quick_output_region_to_firefly(
    all_halos,
    fields_to_include=['particle_mass','particle_radius'],
    fields_units=['log(msun)','kpc'],
    write_files=1)

detected (but did not request) halos Eax
detected (but did not request) halos Eay
detected (but did not request) halos Eaz
detected (but did not request) halos Ebx
detected (but did not request) halos Eby
detected (but did not request) halos Ebz
detected (but did not request) halos Ecx
detected (but did not request) halos Ecy
detected (but did not request) halos Ecz
detected (but did not request) halos Ekin
detected (but did not request) halos Epot
detected (but did not request) halos ID
detected (but did not request) halos Lx
detected (but did not request) halos Ly
detected (but did not request) halos Lz
detected (but did not request) halos Mvir
detected (but did not request) halos Phi0
detected (but did not request) halos Rmax
detected (but did not request) halos Rvir
detected (but did not request) halos SurfP
detected (but did not request) halos VXc
detected (but did not request) halos VYc
detected (but did not request) halos VZc
detected (but did not request) halos Vmax
detected (b